In [18]:
# Detect current folder to avoid package import error

import os, sys
currentdir = os.path.dirname(os.path.realpath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(currentdir)

In [19]:
# Import Opensource packages

import numpy as np
import pandas as pd

import idaes
import pyomo.environ as pyo

from pyomo.core.util import quicksum 
from pyomo.core.expr import evaluate_expression

## Create Concrete Model

In [20]:
from pyomo.environ import *
#from mydata import *

model = ConcreteModel(name = "Price Optimization")

In [22]:
# Define dictionary

Data_dict = {
    'sites': ['NO', 'DE'],
    'time_periods': ['Q4'],
    
    # 'cost_total': {
        # ('NO', 'Q4'): 1.32,
        # ('DE', 'Q4'): 1.05,
        # },
        
    # total cost: (cost_energy * d + cost_revshare * d + cost_fixelec + cost_demand + cost_leasing + cost_sm)
    
    # Variable cost
    
    'cost_energy': {
        ('NO', 'Q4'): 0.21,
        ('DE', 'Q4'): 0.31,
        },
    
    'cost_revshare': {
        ('NO', 'Q4'): 0.05,
        ('DE', 'Q4'): 0.07,
        },

    # Fixed cost
    
    'cost_fixelec': {
        ('NO', 'Q4'): 0.1, 
        ('DE', 'Q4'): 0.12,
        },
    
    'cost_demand': {
        ('NO', 'Q4'): 0.03,
        ('DE', 'Q4'): 0.05,
        },
    
    'cost_leasing': {
        ('NO', 'Q4'): 0.02,
        ('DE', 'Q4'): 0.04,
        },
    
    'cost_sm': {
        ('NO', 'Q4'): 0.05,
        ('DE', 'Q4'): 0.07,
        },
    
    'p_min': 0.1,
    'p_max': 0.9,
    'd_max': 100,
    'd_0': 1.73,
    'd_p': 1.97,
}

# Define sets

model.S = Set(initialize=Data_dict['sites'])
model.T = Set(initialize=Data_dict['time_periods'])

'pyomo.core.base.set.OrderedScalarSet'>) on block Price Optimization with a
new Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
This is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.set.OrderedScalarSet'>) on block Price Optimization with a
new Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
This is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [23]:
model.p = Var(
    model.S, 
    model.T,  
    bounds=(Data_dict['p_min'], Data_dict['p_max']),
    domain=NonNegativeReals,
    doc = 'dv: price'
)

### Define Parameters

In [24]:
model.p_min = Param(initialize=Data_dict['p_min'])
model.p_max = Param(initialize=Data_dict['p_max'])
model.d_max = Param(initialize=Data_dict['d_max'])
model.d_0 = Param(initialize=Data_dict['d_0'], within=NonNegativeReals)
model.d_p = Param(initialize=Data_dict['d_p'], within=NonNegativeReals)

# Define the demand function based on the price --> e.g. d(p) = 1 - p

def demand_rule(model, s, t):
    return model.d_0 - model.d_p * model.p[s, t]
model.d = Expression(model.S, model.T, rule=demand_rule)

# Define the total cost calculation

def total_cost_rule(model, s, t):
    return (
        Data_dict['cost_energy'][s,t] * model.d[s,t] +
        Data_dict['cost_revshare'][s,t] * model.d[s,t] +
        Data_dict['cost_fixelec'][s,t] +
        Data_dict['cost_demand'][s,t] +
        Data_dict['cost_leasing'][s,t] +
        Data_dict['cost_sm'][s,t]
    )
model.c = Expression(model.S, model.T, rule=total_cost_rule)

In [25]:
# Define objective

def obj_rule(model):
    return sum(
        model.d[s,t] * (model.p[s,t]-model.c[s,t])
            for s in model.S for t in model.T
        )
model.profit = Objective(rule=obj_rule, sense=maximize)

# Alternative

#model.profit = Objective(
    #expr = sum(
        #model.d[s,t]*(model.p[s,t]-model.c[s,t]) \
            #for s in model.S for t in model.T
        #), 
    #sense = maximize
#)

In [26]:
# Define constraints

def price_constraint_rule(model, s, t):
    return inequality(model.p[s,t], model.p_min, model.p_max)
model.price_constraint = Constraint(model.S, model.T, rule=price_constraint_rule)

def demand_constraint_rule(model, s, t):
    return model.d[s,t] <= model.d_max
model.demand_constraints = Constraint(model.S, model.T, rule=demand_constraint_rule)

In [30]:
import os
os.environ["PYOMO_EXE"] = "/Programme/CPLEX_Studio2211/cplex/bin/x86-64_osx/cplex"
print(os.environ.get("PYOMO_EXE"))

/Programme/CPLEX_Studio2211/cplex/bin/x86-64_osx/cplex


## Solving the Model

In [29]:
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

# Create a solver instance
solver = SolverFactory('cplex')

# Solve the model
result = solver.solve(model)

# Check the solver status
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    # The model was solved to optimality
    print("Optimal solution found.")
    # Access the variable values
    for var in model.component_data_objects(Var):
        print(f"{var.name}: {var.value}")
else:
    # The model failed to solve
    print("Solver did not find an optimal solution.")

cplex


ApplicationError: No executable found for solver 'cplex'